<h1>Clustering (MESSY)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

titles = df.title.tolist()
summaries = df.gensim_summary.tolist()

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(summaries)



true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)


pred_clusters = model.predict(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
#plt.scatter(X[:, 0], X[:, 1], c=pred_clusters, s=10)
terms = vectorizer.get_feature_names()
plt.scatter(order_centroids[:, 0], order_centroids[:, 1], c='black', s=70)

import ast

len(ast.literal_eval(test))

In [ ]:
def cluster_article_sents(sentences):
    #tune function for edge cases
    if type(sentences) == str:
        #if text is a list within a string:
        if sentences[0] == '[':
            #perform literal eval of string (take off the quotes and keep the list)
            document = ast.literal_eval(sentences)
            print('literal eval')
        else:
            #if raw text, tokenize the sentences
            document = sent_tokenize(sentences, language = 'en')
            print('raw text')
    else:
        document = sentences
        print('already tokenize')
    print(document[:10])
    print(len(document))
    
    #vectorize sentences of the document
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(document)
    
    #cluster sentences within document
    k = int(np.floor(len(document)**.5))
    print(k)
    model = KMeans(n_clusters = k, random_state = 0) #, init = 'k-means++', max_iter = 100, n_init = 1)
    
    #fit model to document
    model.fit(X)
   
    #create empty lists to hold ################
    avg = []
    closest = []
    
    for i in range(k):
        idx = np.where(model.labels_ == i)[0]
        print(idx)
        avg.append(np.mean(idx))
    print(avg)
    print(model.cluster_centers_)
    #closest, _ = pairwise_distances_argmin_min(model.cluster_centers_, !!!!)

    
    
    #plotting
    centers = model.cluster_centers_ #.argsort()[:, ::-1]
    #plt.scatter(X[:, 0], X[:, 1], c = X, s=10)
    terms = vectorizer.get_feature_names()
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=70)

In [ ]:
test = df.sent_tokenized[2]
len(ast.literal_eval(test))

In [ ]:
sample = ['this is one sentence.', 'this is a second sentence.', 'this is a third.']

In [ ]:
cluster_article_sents(test)

<h1> LDA

In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                        # minimum reqd occurences of a word 
                             stop_words= stops,             # remove stop words
                             #token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(df['lemmatize_first_100'])

#materialize the sparse data
data_dense = data_vectorized.todense()

#compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

# Build LDA Model
lda_model = LatentDirichletAllocation(n_topics=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

def show_topics(vectorizer, lda_model, n_words=10):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

df.lemmatize_first_100[0]

show_topics(vectorizer, lda_model, n_words=5)